In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple

import requests
import os

# Set your OpenAI API key and API base here
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

def get_movie_plot(movie_name):
    headers = {
        'User-Agent': 'MoviePlotFetcher/1.0'
    }
    
    base_url = f"https://en.wikipedia.org/w/api.php"
        
    def is_movie_page(title):
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "categories|revisions",
            "rvprop": "content",
            "cllimit": "max"
        }
    
        response = requests.get(base_url, headers=headers, params=params)
        data = response.json()
    
        try:
            page = list(data["query"]["pages"].values())[0]
            
            # Check categories for Movie indication
            categories = [cat["title"] for cat in page.get("categories", [])]
            for category in categories:
                if "films" in category.lower():
                    return True
                    
            # Check for infobox movie in the page content
            content = page["revisions"][0]["*"]
            if "{{Infobox film" in content:
                return True
                
        except Exception:
            pass

        return False
    
    def extract_plot_from_text(full_text):
        try:
            # Find the start of the Plot section
            plot_start = full_text.index("== Plot ==") + len("== Plot ==")
            
            # Find the start of the next section
            next_section_start = full_text.find("==", plot_start)

            # If no next section is found, use the end of the text
            if next_section_start == -1:
                next_section_start = len(full_text)

            # Extract the plot text and strip leading/trailing whitespace
            plot_text = full_text[plot_start:next_section_start].strip()

            # Return the extracted plot
            return plot_text

        except ValueError:
            # Return a message if the Plot section isn't found
            return "Plot section not found in the text."
        
    def extract_first_paragraph(full_text):
        # Find the first double newline
        end_of_first_paragraph = full_text.find("\n\n")

        # If found, slice the string to get the first paragraph
        if end_of_first_paragraph != -1:
            return full_text[:end_of_first_paragraph].strip()

        # If not found, return the whole text as it might be just one paragraph
        return full_text.strip()

    
    search_params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": movie_name,
        "utf8": 1,
        "srlimit": 5  # Top 5 search results
    }

    response = requests.get(base_url, headers=headers, params=search_params)
    data = response.json()
    
    # Go through top search results to find a movie page
    for search_result in data.get("query", {}).get("search", []):
        title = search_result["title"]
        if is_movie_page(title):
            # Fetch plot for the movie page
            plot_params = {
                "action": "query",
                "format": "json",
                "titles": title,
                "prop": "extracts",
                "explaintext": True,
            }
            
            plot_response = requests.get(base_url, headers=headers, params=plot_params)
            plot_data = plot_response.json()
            
            try:
                page = list(plot_data["query"]["pages"].values())[0]
                full_text = page.get("extract", "No text...")
                return f"""Overview:\n{extract_first_paragraph(full_text)}\nPlot:\n{extract_plot_from_text(full_text)}""".strip()
            except:
                return "Error fetching plot."

    return "Movie not found."


model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=1000)

# Interactively ask personal questions
personal_questions = [
    "Which movie genre do you like the most?",
    "What is your favorite color?",
    "What is your favorite movie?",
    "Pick one - dogs, cats or hamsters?",
    "What is your favorite food?",
    "What is your favorite drink?"
]

personal_answers = []
print("Please answer the following personal questions to help build your persona:")
for q in personal_questions:
    ans = input(q + " ")
    personal_answers.append(ans)

# Ask user for movies to consider
movies = []
num_movies = int(input("How many movies are you considering? "))
for _ in range(num_movies):
    movie_input = input("Enter a movie title: ")
    movies.append(movie_input)

max_rating = 100

history = ChatMessageHistory()
history.add_user_message(f"You are an AI that will recommend a movie based on my answers to personal questions. Ask me {len(personal_questions)} questions.")
for i in range(len(personal_questions)):
    # Add AI's question and user's answer to the history
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
history.add_ai_message("Now tell me a plot summary of a movie you're considering watching, and specify how you want me to respond to you with the movie rating")

summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="recommendation_summary", 
    input_key="input",
    buffer=f"The human answered {len(personal_questions)} personal questions). Use them to rate, from 1 to {max_rating}, how much they like a movie they describe to you.",
    return_messages=True
)

# Custom memory class as defined in the original code
class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
    
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers", 
    input_key="input"
)

# Combine memories
memory = CombinedMemory(memories=[conversational_memory, summary_memory])

RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Movie Recommender. 
The AI follows human instructions and provides movie ratings for a human based on the movie plot
and the human's persona derived from their answers.

Summary of Recommendations:
{recommendation_summary}
Personal Questions and Answers:
{questions_and_answers}
Human: {input}
AI:"""

PROMPT = PromptTemplate(
    input_variables=["recommendation_summary", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)

recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)

# Rate each movie based on the user persona and movie plot
for movie in movies:
    print("Considering movie: " + movie)
    movie_plot = get_movie_plot(movie)
    
    plot_rating_instructions = f"""
         =====================================
        === START MOVIE PLOT SUMMARY FOR {movie} ===
        {movie_plot}
        === END MOVIE PLOT SUMMARY ===
        =====================================
        
        RATING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
        AI will provide a highly personalized rating based only on the movie plot summary human provided 
        and human answers to questions included with the context. 
        AI should be very sensible to human personal preferences captured in the answers to personal questions, 
        and should not be influenced by anything else.
        AI will also build a persona for the human based on their answers to questions, and use this persona to rate the movie.
        OUTPUT FORMAT:
        First, include that persona you came up with in the explanation for the rating. Describe the persona in a few sentences.
        Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
        In addition, consider other ratings for this human that you might have as they might give you more information about human's preferences.
        Your goal is to provide a rating that is as close as possible to the rating the human would give to this movie.
        Remember that the human has very limited time and wants to see something they will like, so your rating should be as accurate as possible.
        Rating will range from 1 to {max_rating}, with {max_rating} meaning human will love it, and 1 meaning human will hate it. 
        You will include a logical explanation for your rating based on the human persona you've built and their responses.
        YOUR REVIEW MUST END WITH TEXT: "RATING FOR MOVIE {movie} is " FOLLOWED BY THE RATING.
        FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO UNDERSTAND YOUR REVIEW.
    """
    prediction = recommender.predict(input=plot_rating_instructions)
    print(prediction)

# After rating all movies, choose the best one
final_recommendation = """
Now that AI has rated all the movies, AI will recommend the one that the human will like the most.
AI will respond with a movie recommendation and a short explanation of why the human will like it over all other movies. 
AI will not include any ratings in the explanation, only the reasons why the human will like it the most. 
However, the movie must be one of the previously rated movies. The chosen movie should be the one that received the highest rating.
For example, if one movie was rated 65, and another 60, recommend the one with rating 65.
"""
recommendation_result = recommender.predict(input=final_recommendation)
print(recommendation_result)


/tmp/ipykernel_8712/2224601200.py:129: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=1000)


Please answer the following personal questions to help build your persona:
Considering movie: Event Horizon


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Movie Recommender. 
The AI follows human instructions and provides movie ratings for a human based on the movie plot
and the human's persona derived from their answers.

Summary of Recommendations:
[SystemMessage(content='The human answered 6 personal questions). Use them to rate, from 1 to 100, how much they like a movie they describe to you.', additional_kwargs={}, response_metadata={})]
Personal Questions and Answers:
Human: You are an AI that will recommend a movie based on my answers to personal questions. Ask me 6 questions.
AI: Which movie genre do you like the most?
Human: Science Fiction and Mystery
AI: What is your favorite color?
Human: green
AI: What is your favorite movie?
Human: Dune
AI: Pick one - dogs, cats or hamsters?
Human: cat

It seems the search in wikipedia has confused the movie which I entered (Dead Pool 3) with Infinity Pool! But this is not a fault of the AI but of the wikipedia search.